In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110610  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,4.197739e+06
2018-02-28,3.844915e+06
2018-03-31,3.963245e+06
2018-04-30,3.955367e+06
2018-05-31,4.110645e+06
2018-06-30,3.959789e+06
2018-07-31,4.524809e+06
2018-08-31,4.352554e+06
2018-09-30,3.825835e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    4.197739e+06
 2018-02-28    3.844915e+06
 2018-03-31    3.963245e+06
 2018-04-30    3.955367e+06
 2018-05-31    4.110645e+06
 2018-06-30    3.959789e+06
 2018-07-31    4.524809e+06
 2018-08-31    4.352554e+06
 2018-09-30    3.825835e+06
 2018-10-31    3.720514e+06
 2018-11-30    3.709846e+06
 2018-12-31    3.929811e+06
 2019-01-31    4.219129e+06
 2019-02-28    3.709218e+06
 2019-03-31    3.842791e+06
 2019-04-30    3.705060e+06
 2019-05-31    3.931999e+06
 2019-06-30    3.649177e+06
 2019-07-31    4.249172e+06
 2019-08-31    4.154667e+06
 2019-09-30    3.602306e+06
 2019-10-31    3.647831e+06
 2019-11-30    3.287087e+06
 2019-12-31    5.047524e+06
 2020-01-31    4.918793e+06
 2020-02-29    5.124987e+06
 2020-03-31    3.759727e+06
 2020-04-30    3.412218e+06
 2020-05-31    3.747153e+06
 2020-06-30    3.654600e+06
 2020-07-31    4.120128e+06
 2020-08-31    4.314032e+06
 2020-09-30    3.781302e+06
 2020-10-31    4.035863e+06
 2020-11-30    3.973692e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -5.741700
p-value : 0.000001
Critical Values :
	1%: -3.610400
	5%: -2.939109
	10%: -2.608063


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1201.686, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1199.607, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1197.525, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1196.488, Time=0.07 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1193.473, Time=0.12 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1196.097, Time=0.14 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1199.700, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1201.448, Time=0.10 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1198.927, Time=0.06 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1194.741, Time=0.20 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1195.113, Time=0.30 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1200.188, Time=0.06 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1204.647, Time=0.26 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1195.885, Time=0.18 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1195.113, Time=0.32 sec
 ARIMA(3,0

ARIMA(order=(3, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(3,0,0))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(3, 0)   Log Likelihood                -591.165
Method:                       css-mle   S.D. of innovations         310752.582
Date:                Sat, 11 Sep 2021   AIC                           1192.329
Time:                        02:05:27   BIC                           1201.018
Sample:                             0   HQIC                          1195.514
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.993e+06   4.11e+04     97.255      0.000    3.91e+06    4.07e+06
ar.L1.y        0.2677      0.134      1.993      0.046       0.004       0.531
ar.L2.y       -0.0171      0.140     -0.123      0.9

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([4055181.16425163, 3975680.14902469]), array([310752.58209858, 321697.33869553]), array([[3446117.29523558, 4664245.03326768],
       [3345164.95125907, 4606195.34679032]]))
